In [35]:
#Import the necessary libraries

import os
import py2exe
import pickle
import pathlib
import webbrowser
import matplotlib
import numpy as np
import pandas as pd
import tkinter as tk
import seaborn as sns
import pyswarms as ps
from PIL import Image
from pathlib import Path
from itertools import cycle
from sklearn import metrics
from numpy import genfromtxt
from subprocess import Popen
import statsmodels.api as sm
import tkinter.font as tkFont
import matplotlib.pyplot as plt
from matplotlib import rcParams
from distutils.core import setup
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from pykrige.ok import OrdinaryKriging
from sklearn.metrics import roc_curve, auc
from matplotlib.colors import ListedColormap
import matplotlib.transforms as mtransforms
from gplearn.genetic import SymbolicClassifier
from tkinter import ttk, filedialog, messagebox
from sklearn.preprocessing import label_binarize
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from matplotlib.pyplot import Figure, MultipleLocator
from sklearn.model_selection import train_test_split, KFold
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from sklearn.metrics import brier_score_loss, confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score, balanced_accuracy_score

config = {"font.family":'Times New Roman',  "font.size": 10,}
rcParams.update(config)
tfd = tfp.distributions

In [37]:
#Define the main menu

class Main_menu(tk.Tk):
    def __init__(self):
        super().__init__() 
        self.createWidgets()

    def createWidgets(self):
        self.title('Stope stability prediction')
        self.columnconfigure(0, minsize=200)
        self.geometry('750x280')
        topframe = tk.Frame(self, height=20)
        topframe.pack(fill="both", expand="yes", side=tk.TOP)
        path = pathlib.Path('__file__').parent.absolute()
        path_main_interface = os.path.join(path, 'Interface.png')
        self.Main_interface_image = tk.PhotoImage(file=path_main_interface)
        Main_interface = tk.Label(topframe, image=self.Main_interface_image)
        Main_interface.place(x=0, y=0, relwidth=1, relheight=1)
        self.menubar = tk.Menu()
        configuration = tk.Menu(self.menubar, tearoff=0)
        configuration.add_command(label="Machine learning model", command=self.config1)
        self.menubar.add_cascade(label="Function selection", menu=configuration)
        self.menubar.add_cascade(label="Quit", command=quit)
        self.config(menu=self.menubar)   
        
    def config1(self):
        self.destroy()
        ML_technology()

In [39]:
#Stope stability prediction based on the optimized MLP classifier

class ML_technology(tk.Tk):
    def __init__(self):
        super().__init__()
        self.createWidgets()
        
    def createWidgets(self):
        self.title('Stope stability prediction based on the optimized MLP classifier')
        self.columnconfigure(0, minsize=100)
        self.geometry('750x780') 
        global clicked
        clicked = False
        
        fir_font = tkFont.Font(family="Times new roman", size=15, weight="normal")
        sec_font = tkFont.Font(family="Times new roman", size=14, weight="normal")
        sec_font2 = tkFont.Font(family="Times new roman", size=14, weight="normal")   
        thi_font = tkFont.Font(family="Times new roman", size=27, weight="bold")
        fou_font = tkFont.Font(family="Times new roman", size=16, weight="normal")
        fiv_font = tkFont.Font(family="Times new roman", size=15, weight="normal")
        
        topframe = tk.Frame(self, height=80)
        topframe.pack(fill="both", expand="yes", side=tk.TOP)
        inputframe = tk.Frame(self, height=60)
        inputframe.pack(fill="both", expand="yes", side=tk.TOP)
        predictframe = tk.Frame(self, height=80)
        predictframe.pack(fill="both", expand="yes", side=tk.TOP)
        Visualizationframe = tk.Frame(self, height=80)
        Visualizationframe.pack(fill="both", expand="yes", side=tk.TOP)
        
        labelframe1 = tk.LabelFrame(topframe, font=fir_font,background="#C0C0C0")
        labelframe1.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        labelframe2 = tk.LabelFrame(inputframe, text="Initialization", font=fir_font,background="#F0FFF0")
        labelframe2.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        labelframe3 = tk.LabelFrame(inputframe, text="Training performance", font=fir_font,background="#F0FFF0")
        labelframe3.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        labelframe4 = tk.LabelFrame(inputframe, text="Test performance", font=fir_font,background="#F0FFF0")
        labelframe4.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        labelframe5 = tk.LabelFrame(predictframe, text="Input feature", font=fir_font,background="#FFFACD")
        labelframe5.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        labelframe6 = tk.LabelFrame(predictframe, text="Stability level", font=fir_font,background="#FFFACD")
        labelframe6.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        labelframe7 = tk.LabelFrame(Visualizationframe, text="Update the stability graph", font=fiv_font,background="#F0FFF0")
        labelframe7.pack(side=tk.LEFT, fill=tk.Y, expand="yes")
        
        self.entryvar = tk.StringVar()
        tk.Label(labelframe1, text='Path：', font=fir_font).grid(row=0, column=0, sticky=tk.W)
        tk.Entry(labelframe1, width=32,bd=4,textvariable=self.entryvar).grid(row=0, column=1)
        tk.Button(labelframe1, command=self.__openfile, text='Dataset loading', font=fir_font). \
            grid(row=0, column=2, padx=10, pady=10)
        tk.Button(labelframe1, command=self.Training, text='Modeling', font=fir_font) \
            .grid(row=0, column=3, padx=10, pady=10)
        tk.Button(labelframe1, command=self.Predict, text='Predict', font=fir_font) \
            .grid(row=0, column=4, padx=10, pady=10)
        tk.Button(labelframe1, command=self.Main_interface, text='Quit', font=fir_font) \
            .grid(row=0, column=5, padx=10, pady=10)
        
        self.MLP_Test_size_Parameter = tk.StringVar()
        self.MLP_Validation_size_Parameter = tk.StringVar()
        self.MLP_hidden_layer_sizes_Parameter = tk.StringVar()
        self.MLP_alpha_Parameter = tk.StringVar()
        self.MLP_Random_State_Parameter = tk.StringVar()
        
        ent_Test_size=tk.Entry(labelframe2, width=10, textvariable=self.MLP_Test_size_Parameter, validate='key', fg='gray', font=sec_font)
        ent_Test_size.grid(row=1, column=2, padx=4, pady=10)
        ent_Test_size.insert(0, '0.20')
        self.Default_hint(ent_Test_size)
        ent_hidden_layer_sizes=tk.Entry(labelframe2, width=10, textvariable=self.MLP_hidden_layer_sizes_Parameter, validate='key', fg='gray', font=sec_font)
        ent_hidden_layer_sizes.grid(row=2, column=2, padx=4, pady=10)
        ent_hidden_layer_sizes.insert(0, '106')
        self.Default_hint(ent_hidden_layer_sizes)
        ent_alpha=tk.Entry(labelframe2, width=10, textvariable=self.MLP_alpha_Parameter, validate='key', fg='gray', font=sec_font)
        ent_alpha.grid(row=3, column=2, padx=4, pady=10)
        ent_alpha.insert(0, '0.09086556279101839')
        self.Default_hint(ent_alpha)
        ent_MLP_Random_State=tk.Entry(labelframe2, width=10, textvariable=self.MLP_Random_State_Parameter, validate='key', fg='gray', font=sec_font)
        ent_MLP_Random_State.grid(row=4, column=2, padx=4, pady=10)
        ent_MLP_Random_State.insert(0, '0')
        self.Default_hint(ent_MLP_Random_State)
        
        lab_Test_size=tk.Label(labelframe2, text='Dataset split ratio', font=sec_font)
        lab_Test_size.grid(row=1, column=1, sticky='w', padx=4, pady=10)
        self.CreateToolTip(lab_Test_size,text='The proportion of test sets in the input data set')
        lab_hidden_layer_sizes=tk.Label(labelframe2, text='Hidden layer sizes', font=sec_font)
        lab_hidden_layer_sizes.grid(row=2, column=1, sticky='w', padx=4, pady=10)
        self.CreateToolTip(lab_hidden_layer_sizes, text='The number of neurons in each hidden layer of a neural network')
        lab_alpha=tk.Label(labelframe2, text='Parameter alpha', font=sec_font)
        lab_alpha.grid(row=3, column=1, sticky='w', padx=4, pady=10)
        self.CreateToolTip(lab_alpha, text='Affects the complexity of the model')
        lab_MLP_Random_State=tk.Label(labelframe2, text='Random number', font=sec_font)
        lab_MLP_Random_State.grid(row=4, column=1, sticky='w', padx=4, pady=10)
        self.CreateToolTip(lab_MLP_Random_State, text='The number of random seeds in the MLP model')
        
        self.ACC_train = tk.StringVar()
        self.BACC_train = tk.StringVar()
        self.PRE_train = tk.StringVar()
        self.REC_train = tk.StringVar()
        self.F1W_train = tk.StringVar()
        self.ACC_test = tk.StringVar()
        self.BACC_test = tk.StringVar()
        self.PRE_test = tk.StringVar()
        self.REC_test = tk.StringVar()
        self.F1W_test = tk.StringVar()
        
        tk.Entry(labelframe3, width=10, textvariable=self.ACC_train, validate='key', state=tk.DISABLED, font=sec_font2).grid(row=1,column=5, padx=5, pady=5)
        tk.Entry(labelframe3, width=10, textvariable=self.BACC_train, validate='key', state=tk.DISABLED, font=sec_font2).grid(row=2,column=5, padx=5, pady=5)
        tk.Entry(labelframe3, width=10, textvariable=self.PRE_train, validate='key', state=tk.DISABLED, font=sec_font2).grid(row=3,column=5, padx=5, pady=5)
        tk.Entry(labelframe3, width=10, textvariable=self.REC_train, validate='key', state=tk.DISABLED, font=sec_font2).grid(row=4,column=5, padx=5, pady=5)
        tk.Entry(labelframe3, width=10, textvariable=self.F1W_train, validate='key', state=tk.DISABLED, font=sec_font2).grid(row=5, column=5, padx=5, pady=5)

        tk.Label(labelframe3, text='Accuracy', font=sec_font).grid(row=1, column=4, sticky='w', padx=5, pady=5)
        tk.Label(labelframe3, text='Balanced ACC', font=sec_font).grid(row=2, column=4, sticky='w', padx=5, pady=5)
        tk.Label(labelframe3, text='Precision', font=sec_font).grid(row=3, column=4, sticky='w', padx=5, pady=5)
        tk.Label(labelframe3, text='Recall', font=sec_font).grid(row=4, column=4, sticky='w', padx=5, pady=5)
        tk.Label(labelframe3, text='Weighted F1', font=sec_font).grid(row=5, column=4, sticky='w', padx=5, pady=5)
        
        tk.Entry(labelframe4, width=10, textvariable=self.ACC_test, validate='key', state=tk.DISABLED, font=sec_font2).grid(row=1,column=6, padx=5, pady=5)
        tk.Entry(labelframe4, width=10, textvariable=self.BACC_test, validate='key', state=tk.DISABLED, font=sec_font2).grid(row=2,column=6, padx=5, pady=5)
        tk.Entry(labelframe4, width=10, textvariable=self.PRE_test, validate='key', state=tk.DISABLED, font=sec_font2).grid(row=3, column=6, padx=5, pady=5)
        tk.Entry(labelframe4, width=10, textvariable=self.REC_test, validate='key', state=tk.DISABLED, font=sec_font2).grid(row=4, column=6, padx=5, pady=5)
        tk.Entry(labelframe4, width=10, textvariable=self.F1W_test, validate='key', state=tk.DISABLED, font=sec_font2).grid(row=5, column=6, padx=5, pady=5)

        tk.Label(labelframe4, text='Accuracy', font=sec_font).grid(row=1, column=5, sticky='w', padx=5, pady=5)
        tk.Label(labelframe4, text='Balanced ACC', font=sec_font).grid(row=2, column=5, sticky='w', padx=5, pady=5)
        tk.Label(labelframe4, text='Precision', font=sec_font).grid(row=3, column=5, sticky='w', padx=5, pady=5)
        tk.Label(labelframe4, text='Recall', font=sec_font).grid(row=4, column=5, sticky='w', padx=5, pady=5)
        tk.Label(labelframe4, text='Weighted F1', font=sec_font).grid(row=5, column=5, sticky='w', padx=5, pady=5)
        
        self.Q_Input = tk.StringVar()
        self.Jn_Input = tk.StringVar()
        self.Ja_Input = tk.StringVar()
        self.A_Input = tk.StringVar()
        self.B_Input = tk.StringVar()
        self.C_Input = tk.StringVar()
        self.HR_Input = tk.StringVar()
        self.N_Input = tk.StringVar()
        self.MLP_Stability_Degree = tk.StringVar()
        
        ent1=tk.Entry(labelframe5, textvariable=self.Q_Input, validate='key', bg='#FFDEAD',width=4,bd=4, font=fou_font,justify='center')
        ent1.grid(row=1, column=2, padx=5, pady=5)
        self.Default_hint(ent1)
        ent2=tk.Entry(labelframe5,  textvariable=self.Jn_Input, validate='key', bg='#FFDEAD',width=4,bd=4,font=fou_font,justify='center')
        ent2.grid(row=2, column=2, padx=5, pady=5)
        self.Default_hint(ent2)
        ent3=tk.Entry(labelframe5, textvariable=self.Ja_Input, validate='key', bg='#FFDEAD',width=4,bd=4, font=fou_font,justify='center')
        ent3.grid(row=1, column=4, padx=5, pady=5)
        self.Default_hint(ent3)
        ent4=tk.Entry(labelframe5,  textvariable=self.A_Input, validate='key', bg='#FFDEAD',width=4,bd=4,font=fou_font,justify='center')
        ent4.grid(row=2, column=4, padx=5, pady=5)
        self.Default_hint(ent4)
        ent5=tk.Entry(labelframe5, textvariable=self.B_Input, validate='key', bg='#FFDEAD',width=4,bd=4, font=fou_font,justify='center')
        ent5.grid(row=1, column=6, padx=5, pady=5)
        self.Default_hint(ent5)
        ent6=tk.Entry(labelframe5,  textvariable=self.C_Input, validate='key', bg='#FFDEAD',width=4,bd=4,font=fou_font,justify='center')
        ent6.grid(row=2, column=6, padx=5, pady=5)
        self.Default_hint(ent6)
        ent7=tk.Entry(labelframe5, textvariable=self.HR_Input, validate='key', bg='#FFDEAD',width=4,bd=4, font=fou_font,justify='center')
        ent7.grid(row=1, column=8, padx=5, pady=5)
        self.Default_hint(ent7)
        ent8=tk.Entry(labelframe5,  textvariable=self.N_Input, validate='key', bg='#FFDEAD',width=4,bd=4,font=fou_font,justify='center')
        ent8.grid(row=2, column=8, padx=5, pady=5)
        self.Default_hint(ent8)
        
        lab1=tk.Label(labelframe5, text='Q`', font=fou_font,bg='#FFDEAD')
        lab1.grid(row=1, column=1, sticky='e', padx=4, pady=5)
        self.CreateToolTip(lab1,text='Q´ represents the modified NGI Q classification index value')
        lab2=tk.Label(labelframe5, text='Jn', font=fou_font,bg='#FFDEAD')
        lab2.grid(row=2, column=1, sticky='e', padx=4, pady=5)
        self.CreateToolTip(lab2,text='Joint set number')
        lab3=tk.Label(labelframe5, text='Ja', font=fou_font,bg='#FFDEAD')
        lab3.grid(row=1, column=3, sticky='e', padx=4, pady=5)
        self.CreateToolTip(lab3,text='Joint alteration number')
        lab4=tk.Label(labelframe5, text='A', font=fou_font,bg='#FFDEAD')
        lab4.grid(row=2, column=3, sticky='e', padx=4, pady=5)
        self.CreateToolTip(lab4,text='Ratio of the unconfined compressive strength of the intact rock to the induced compressive stress parallel to the stope face')
        lab5=tk.Label(labelframe5, text='B', font=fou_font,bg='#FFDEAD')
        lab5.grid(row=1, column=5, sticky='e', padx=4, pady=5)
        self.CreateToolTip(lab5,text='Orientation adjustment factor')
        lab6=tk.Label(labelframe5, text='C', font=fou_font,bg='#FFDEAD')
        lab6.grid(row=2, column=5, sticky='e', padx=4, pady=5)
        self.CreateToolTip(lab6,text='Gravity adjustment factor')
        lab7=tk.Label(labelframe5, text='HR', font=fou_font,bg='#FFDEAD')
        lab7.grid(row=1, column=7, sticky='e', padx=4, pady=5)
        self.CreateToolTip(lab7,text='Ratio of surface area to surface circumference of exposed surface of stope')
        lab8=tk.Label(labelframe5, text='N', font=fou_font,bg='#FFDEAD')
        lab8.grid(row=2, column=7, sticky='e', padx=4, pady=5)
        self.CreateToolTip(lab8, text='A comprehensive index used to evaluate the stability of rock mass under given stress conditions')

        tk.Entry(labelframe6, width=16, textvariable=self.MLP_Stability_Degree, validate='key', state=tk.DISABLED,font=thi_font,justify='center').grid(row=1, column=4, padx=20, pady=20) 

        self.MLP_fig = Figure(figsize=(7.3, 3.5), dpi=100,frameon=True,edgecolor = "#FFF5EE")
        self.MLP_canvas = FigureCanvasTkAgg(self.MLP_fig, master=labelframe7)
        self.MLP_canvas.get_tk_widget().grid(column = 1, row = 1)
        self.MLP_canvas._tkcanvas.pack(side=tk.TOP, fill=tk.X, expand=1)
        self.MLP_ax = self.MLP_fig.add_subplot(111)
        self.MLP_fig.tight_layout(h_pad=2,w_pad=2)

    def Main_interface(self):
        self.destroy()
        Main_menu()

    def Default_hint(self, widget):
        global clicked
        def hint(event):
            if (clicked == False):
                widget.delete(0, tk.END)
                widget.config(fg="black")
        widget.bind("<Button-1>", hint)

    def CreateToolTip(self, widget, text):
        toolTip = ToolTip(widget)
        def enter(event):
            toolTip.showtip(text)
        def leave(event):
            toolTip.hidetip()
        widget.bind('<Enter>', enter)
        widget.bind('<Leave>', leave)

    def __openfile(self):
        self.filename = filedialog.askopenfilename(title='open csv file', filetype=[('csv', '*.csv')])
        self.entryvar.set(self.filename)
        if not self.filename:
            messagebox.showwarning('Warning', message='File not yet selected！')

    def Training(self):
        dataset = pd.read_csv(self.filename) 
        MLP_Test_size=float(self.MLP_Test_size_Parameter.get())
        MLP_hidden_layer_sizes=int(self.MLP_hidden_layer_sizes_Parameter.get())
        MLP_alpha=float(self.MLP_alpha_Parameter.get())
        MLP_Random_State=int(self.MLP_Random_State_Parameter.get())
        np.random.seed(MLP_Random_State)
        
        X=dataset.iloc[:,0:8]
        y=dataset.iloc[:,8]
        np.random.seed(MLP_Random_State)
        X_Boundary=dataset.iloc[:,6:8].copy()
        y_Boundary=dataset.iloc[:,8].astype(int)
        X_Train, X_test, y_Train, y_test = train_test_split(X,y,test_size=MLP_Test_size)
        X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train,test_size=MLP_Test_size)

        np.random.seed(MLP_Random_State)
        self.MLP_n_samples, self.MLP_n_features = X.shape
        MLP_model = MLPClassifier(hidden_layer_sizes=int(MLP_hidden_layer_sizes,), activation='relu', solver='adam',
                                        alpha=MLP_alpha,batch_size='auto', learning_rate='constant',learning_rate_init=0.001,
                                        power_t=0.5, max_iter=2000, shuffle=False,tol=1e-4, verbose=False, warm_start=False,
                                        momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1,
                                        beta_1=0.9, beta_2=0.999, epsilon=1e-8, n_iter_no_change=10,random_state=None)
        MLP_model.fit(X_train, y_train)
        train_PredictY= MLP_model.predict(X_train)
        trainY_True = y_train
        test_PredictY = MLP_model.predict(X_test)
        testY_True = y_test
        self.test_result = np.hstack((testY_True, test_PredictY))
        train_ACC = accuracy_score(trainY_True, train_PredictY)
        train_BACC = balanced_accuracy_score(trainY_True, train_PredictY)
        train_PRE = precision_score(trainY_True, train_PredictY,pos_label=0)
        train_REC = recall_score(trainY_True, train_PredictY,pos_label=0)
        train_F1W = f1_score(trainY_True, train_PredictY, average='weighted', zero_division=0)
        
        test_ACC = accuracy_score(testY_True, test_PredictY)
        test_BACC = balanced_accuracy_score(testY_True, test_PredictY)
        test_PRE = precision_score(testY_True, test_PredictY,pos_label=0)
        test_REC = recall_score(testY_True, test_PredictY,pos_label=0)
        test_F1W = f1_score(testY_True, test_PredictY, average='weighted', zero_division=0)
        
        self.ACC_train.set(str(train_ACC))
        self.BACC_train.set(str(train_BACC))
        self.PRE_train.set(str(train_PRE))
        self.REC_train.set(str(train_REC))
        self.F1W_train.set(str(train_F1W))
        
        self.ACC_test.set(str(test_ACC))
        self.BACC_test.set(str(test_BACC))
        self.PRE_test.set(str(test_PRE))
        self.REC_test.set(str(test_REC))
        self.F1W_test.set(str(test_F1W))
        
        np.random.seed(MLP_Random_State)
        X_train_Boundary, X_test_Boundary, y_train_Boundary, y_test_Boundary = train_test_split(
            X_Boundary, y_Boundary, test_size=MLP_Test_size, shuffle=True, random_state=None)
        X_train_Boundary, X_val_Boundary, y_train_Boundary, y_val_Boundary = train_test_split(
            X_train_Boundary, y_train_Boundary, test_size=MLP_Test_size, shuffle=True, random_state=None)
        MLP_model_Boundary = MLPClassifier(
            hidden_layer_sizes=(MLP_hidden_layer_sizes,), activation='relu', solver='adam',
            alpha=MLP_alpha, learning_rate='constant',learning_rate_init=0.001, max_iter=2000,shuffle=False,random_state=None,early_stopping=False, verbose=False,beta_1=0.9, beta_2=0.999, epsilon=1e-8, n_iter_no_change=10,)
        MLP_model_Boundary.fit(X_train_Boundary, y_train_Boundary)
  
        def _clip(p, eps=1e-6):
            p = np.asarray(p)
            return np.clip(p, eps, 1 - eps)
        def _logit(p):
            p = _clip(p);  return np.log(p) - np.log(1 - p)
        def _rates_at_tau(y_true, proba, tau):
            y_true = np.asarray(y_true).astype(int)
            y_hat  = (np.asarray(proba) >= tau).astype(int)
            P = max((y_true == 1).sum(), 1); N = max((y_true == 0).sum(), 1)
            TP = np.sum((y_hat == 1) & (y_true == 1)); TN = np.sum((y_hat == 0) & (y_true == 0))
            TPR = TP / P; TNR = TN / N;  BA = 0.5 * (TPR + TNR)
            return BA
        def pick_tau_max_BA(y_true, proba, grid=2001):
            taus = np.linspace(0, 1, grid)
            BA_vals = np.array([_rates_at_tau(y_true, proba, t) for t in taus])
            i = int(np.argmax(BA_vals))
            return float(taus[i]), float(BA_vals[i])
  

        p_all = MLP_model_Boundary.predict_proba(X_Boundary)[:, 1]
        r_all = (y_Boundary.values - p_all).astype(float)
        xk = np.log10(np.clip(X_Boundary["HR"].values, 1e-12, None))
        yk = np.log10(np.clip(X_Boundary["N"].values,  1e-12, None))
    
        cand = ['spherical', 'exponential', 'gaussian']
        kf = KFold(n_splits=5, shuffle=True, random_state=0)
        best_name, best_rmse = None, 1e9
        for name in cand:
            rmses = []
            for tr_idx, va_idx in kf.split(xk):
                ok = OrdinaryKriging(xk[tr_idx], yk[tr_idx], r_all[tr_idx],
                                     variogram_model=name, verbose=False, enable_plotting=False)
                z_pred, _ = ok.execute('points', xk[va_idx], yk[va_idx])
                rmse = float(np.sqrt(np.mean((r_all[va_idx] - np.asarray(z_pred))**2)))
                rmses.append(rmse)
            mean_rmse = float(np.mean(rmses))
            if mean_rmse < best_rmse:
                best_rmse, best_name = mean_rmse, name
    
        ok_final = OrdinaryKriging(xk, yk, r_all, variogram_model=best_name, verbose=False, enable_plotting=False)
    
        def r_points_var(xq, yq):
            z_mean, z_var = ok_final.execute('points', xq, yq)
            return np.asarray(z_mean), np.asarray(z_var)
    
        def r_grid_var(xx, yy):
            Z, V = ok_final.execute('grid', xx, yy)
            return np.asarray(Z), np.asarray(V)

        def pstar_points_lambda(X_df, lmbd):
            p_b = MLP_model_Boundary.predict_proba(X_df)[:, 1]
            xlog = np.log10(np.clip(X_df["HR"].values, 1e-12, None))
            ylog = np.log10(np.clip(X_df["N"].values,  1e-12, None))
            r_m, _ = r_points_var(xlog, ylog)
            r_m = np.clip(r_m, -0.99, 0.99)
            eta = _logit(p_b) + lmbd * r_m
            return 1.0 / (1.0 + np.exp(-eta))
    
        lambdas = np.linspace(0.0, 1.0, 201)
        briers  = [brier_score_loss(y, pstar_points_lambda(X_Boundary, l)) for l in lambdas]
        lambda_star = float(lambdas[int(np.argmin(briers))])
    
        pstar_all = pstar_points_lambda(X_Boundary, lambda_star)
        tau_star, _ = pick_tau_max_BA(y_Boundary, pstar_all, grid=2001)

        hr_min, hr_max = 1.0, 60.0
        n_min,  n_max  = 0.04, 300.0
        Gx, Gy = 400, 400
        HR_grid = np.logspace(np.log10(hr_min), np.log10(hr_max), Gx)
        N_grid  = np.logspace(np.log10(n_min),  np.log10(n_max),  Gy)
        X0, X1  = np.meshgrid(HR_grid, N_grid)
    
        grid_df = pd.DataFrame({"HR": X0.ravel(), "N": X1.ravel()})
        p_base_grid = MLP_model_Boundary.predict_proba(grid_df)[:, 1].reshape(X0.shape)
    
        xg = np.log10(HR_grid); yg = np.log10(N_grid)
        R_mean, R_var = r_grid_var(xg, yg)
        R_mean = np.clip(R_mean, -0.99, 0.99)
        R_var  = np.clip(R_var, 0.0, np.inf)
    
        eta_grid = _logit(p_base_grid) + lambda_star * R_mean
        P_star   = 1.0 / (1.0 + np.exp(-eta_grid))
        SE_p = P_star * (1 - P_star) * abs(lambda_star) * np.sqrt(R_var)
        L = np.clip(P_star - 1.96 * SE_p, 0.0, 1.0)
        U = np.clip(P_star + 1.96 * SE_p, 0.0, 1.0)
    
        uncertain_mask = (L <= tau_star) & (U >= tau_star)
        from scipy.ndimage import label
        
        def clean_uncertainty_mask(mask, min_pixels=300, keep='largest'):
            structure = np.ones((3,3), dtype=int)
            lbl, n = label(mask.astype(np.uint8), structure=structure)
            if n == 0:
                return mask
        
            counts = np.bincount(lbl.ravel())
            if keep == 'largest':
                keep_id = np.argmax(counts[1:]) + 1
                return (lbl == keep_id)
            else:
                keep_ids = np.where(counts >= min_pixels)[0]
                keep_ids = keep_ids[keep_ids != 0]
                return np.isin(lbl, keep_ids)

        uncertain_mask = clean_uncertainty_mask(uncertain_mask, keep='largest')
        uncertain_mask = clean_uncertainty_mask(uncertain_mask, min_pixels=300, keep='by_threshold')

        stable_mask    = (P_star <  tau_star) & (~uncertain_mask)
        unstable_mask  = (P_star >= tau_star) & (~uncertain_mask)

        ax = self.MLP_ax
        ax.cla()

        ax.contourf(X0, X1, stable_mask.astype(int), levels=[0.5, 1.5],
                    colors=['#d0e5f5'], alpha=0.55, zorder=0.50)
        ax.contourf(X0, X1, unstable_mask.astype(int), levels=[0.5, 1.5],
                    colors=['#fee5d9'], alpha=0.55, zorder=0.55)

        red_bins = [(0.4,0.5),(0.5,0.6),(0.6,0.7),(0.7,0.8),(0.8,0.9),(0.9,1.0)]
        red_colors = ['#fff5f0', "#fde0dd","#fcbba1","#fc9272","#fb6a4a","#ef3b2c"]
        for (lo, hi), col in zip(red_bins, red_colors):
            stripe_mask = (P_star >= lo) & (P_star < hi) & (~uncertain_mask) & (P_star >= tau_star)
            if np.any(stripe_mask):
                ax.contourf(X0, X1, stripe_mask.astype(int), levels=[0.5, 1.5],
                            colors=[col], alpha=0.70, zorder=0.60)

        ax.contourf(X0, X1, uncertain_mask.astype(int), levels=[0.5, 1.5],
                    colors=['#c0c0c0'], alpha=1.0, zorder=0.90)

        PROBA_LINES = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
        all_levels = sorted({*(float(f"{lv:.4f}") for lv in PROBA_LINES), float(f"{tau_star:.4f}")})
        colors, linewidths, linestyles = [], [], []
        for lv in all_levels:
            if abs(lv - tau_star) < 1e-6:
                colors.append('#000000'); linewidths.append(2.0); linestyles.append('-')
            elif abs(lv - 0.50) < 1e-12:
                colors.append('#777777'); linewidths.append(1.2); linestyles.append(':')
            else:
                colors.append('#777777'); linewidths.append(1.2); linestyles.append('--')
        cs = ax.contour(X0, X1, P_star, levels=all_levels, colors=colors,
                        linewidths=linewidths, linestyles=linestyles, zorder=0.95)
    
        def _first_crossing_x(level):
            Gy, Gx = P_star.shape
            vals_x = HR_grid
            logx = np.log(vals_x)
            for row in range(Gy):
                row_vals = P_star[row, :]
                diff = row_vals - level
                idxs = np.where(np.signbit(diff[:-1]) != np.signbit(diff[1:]))[0]
                if idxs.size > 0:
                    i = idxs[0]
                    if row_vals[i] == level:   return float(vals_x[i])
                    if row_vals[i+1] == level: return float(vals_x[i+1])
                    y0, y1 = row_vals[i], row_vals[i+1]
                    t = (level - y0) / (y1 - y0)
                    x = np.exp(logx[i] + t * (logx[i+1] - logx[i]))
                    return float(x)
            return None
    
        text_trans = mtransforms.blended_transform_factory(ax.transData, ax.transAxes)
        text_trans = text_trans + mtransforms.ScaledTranslation(0, -8/72, self.MLP_fig.dpi_scale_trans)
        for lv, col in zip(all_levels, colors):
            x_pos = _first_crossing_x(lv)
            if x_pos is None or not (HR_grid.min() <= x_pos <= HR_grid.max()):
                continue
            label = 'τ*' if abs(lv - tau_star) < 1e-6 else f"{lv:.1f}"
            ax.text(x_pos, 0, label, transform=text_trans, ha='center', va='top',
                    fontsize=15, color=('#000' if abs(lv - tau_star) < 1e-6 else col),
                    clip_on=False)

        mask0 = (y_Boundary.values == 0); mask1 = ~mask0
        def s_from_diam_pt(d_pt): return (d_pt/2.0)**2 * np.pi
        S_AREA = s_from_diam_pt(6.0)
        scat0 = ax.scatter(X_Boundary.loc[mask0, "HR"], X_Boundary.loc[mask0, "N"],
                           s=S_AREA, c='#1f77b4', edgecolors='white', linewidths=0.3,
                           alpha=0.90, marker='o', label='Stable', zorder=0.98)
        scat1 = ax.scatter(X_Boundary.loc[mask1, "HR"], X_Boundary.loc[mask1, "N"],
                           s=S_AREA, c='#d62728', edgecolors='white', linewidths=0.3,
                           alpha=0.90, marker='o', label='Unstable', zorder=0.98)

        ax.set_xscale('log'); ax.set_yscale('log')
        ax.set_xlim(HR_grid.min(), HR_grid.max()); ax.set_ylim(N_grid.min(), N_grid.max())
        ax.set_xlabel('Hydraulic Radius (m)', size=12, labelpad=0.5)
        ax.set_ylabel('Stability Number, N', size=10, labelpad=0.5)
        ax.tick_params(direction='in', which='both')
    
        legend_handles = [
            Patch(facecolor='#d0e5f5', label='Stable side (U < τ*)'),
            Patch(facecolor='#ef3b2c', label='Unstable side (L > τ*)'),
            Patch(facecolor='#c0c0c0', alpha=0.50, label='Uncertainty (L ≤ τ* ≤ U)'),
            Line2D([0],[0], color='#000000', lw=2.0, ls='-', label='Decision boundary P*=τ*'),
        ]
        ax.legend(handles=legend_handles, loc='lower right', frameon=True)

        self.ok_final     = ok_final
        self.lambda_star  = lambda_star
        self.tau_star     = tau_star
        self.MLP_model_Boundary = MLP_model_Boundary

        self._input_point_handle = None
        self._input_text_handle  = None

        self.MLP_canvas.draw_idle()
        self.MLP_canvas.get_tk_widget().update_idletasks()

    def _pstar_LU_for_point(self, hr, n):
        df_pt = pd.DataFrame({"HR":[hr], "N":[n]})
        p_base = self.MLP_model_Boundary.predict_proba(df_pt)[0, 1]
        xlog = np.log10(max(hr, 1e-12))
        ylog = np.log10(max(n,  1e-12))
        z_mean, z_var = self.ok_final.execute('points', np.array([xlog]), np.array([ylog]))
        r_mean = float(np.clip(np.asarray(z_mean)[0], -0.99, 0.99))
        r_var  = float(np.clip(np.asarray(z_var)[0],  0.0,   np.inf))
        def _clip(p, eps=1e-6): return np.clip(p, eps, 1 - eps)
        def _logit(p): p=_clip(p); return np.log(p) - np.log(1 - p)
    
        eta   = _logit(p_base) + self.lambda_star * r_mean
        pstar = 1.0 / (1.0 + np.exp(-eta))
        se    = pstar * (1 - pstar) * abs(self.lambda_star) * np.sqrt(r_var)
        L = float(np.clip(pstar - 1.96*se, 0.0, 1.0))
        U = float(np.clip(pstar + 1.96*se, 0.0, 1.0))
        return float(pstar), L, U
    
    
    def _highlight_point_on_axes(self, hr, n, label_text=None, color=None):
        ax = self.MLP_ax
        if self._input_point_handle is not None:
            try: self._input_point_handle.remove()
            except Exception: pass
            self._input_point_handle = None
        if self._input_text_handle is not None:
            try: self._input_text_handle.remove()
            except Exception: pass
            self._input_text_handle = None

        if color is None:
            color = 'k'

        self._input_point_handle = ax.scatter(
            [hr], [n],
            marker='*', s=400,
            facecolors='#000000', edgecolors='white', linewidths=2,
            zorder=1.10
        )

        if label_text:
            self._input_text_handle = ax.annotate(
                label_text,
                xy=(hr, n), xycoords='data',
                xytext=(0, 10), textcoords='offset points',
                ha='center', va='bottom',
                fontsize=13, color='#000000',
                bbox=dict(facecolor='white', edgecolor='#000000', alpha=0.85, boxstyle='round,pad=0.2'),
                zorder=1.12, clip_on=False
            )

        import math
        xlo, xhi = ax.get_xlim(); ylo, yhi = ax.get_ylim()
        new_xlo = min(xlo, hr*0.95) if hr>0 else xlo
        new_xhi = max(xhi, hr*1.05)
        new_ylo = min(ylo, n*0.95)  if n>0  else ylo
        new_yhi = max(yhi, n*1.05)
        new_xlo = max(new_xlo, 1e-8); new_ylo = max(new_ylo, 1e-8)
        ax.set_xlim(new_xlo, new_xhi); ax.set_ylim(new_ylo, new_yhi)
        self.MLP_canvas.draw_idle()
        self.MLP_canvas.get_tk_widget().update_idletasks()

 
    def Predict(self):
        Stability_Graph_Q=float(self.Q_Input.get())
        Stability_Graph_Jn=float(self.Jn_Input.get())
        Stability_Graph_Ja=float(self.Ja_Input.get())
        Stability_Graph_A=float(self.A_Input.get())
        Stability_Graph_B=float(self.B_Input.get())
        Stability_Graph_C=float(self.C_Input.get())
        Stability_Graph_HR=float(self.HR_Input.get())
        Stability_Graph_N=float(self.N_Input.get())
        dataset = pd.read_csv(self.filename)        
        X=dataset.iloc[:,0:8]
        y=dataset.iloc[:,8] 
        MLP_Test_size=float(self.MLP_Test_size_Parameter.get())
        MLP_Random_State=int(self.MLP_Random_State_Parameter.get())
        np.random.seed(MLP_Random_State)
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=MLP_Test_size)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,test_size=MLP_Test_size)
        MLP_hidden_layer_sizes=int(self.MLP_hidden_layer_sizes_Parameter.get())
        MLP_alpha=float(self.MLP_alpha_Parameter.get())
        MLP_model = MLPClassifier(hidden_layer_sizes=int(MLP_hidden_layer_sizes,), activation='relu', solver='adam',
                                        alpha=MLP_alpha,batch_size='auto', learning_rate='constant',learning_rate_init=0.001,
                                        power_t=0.5, max_iter=2000, shuffle=False,random_state=None,tol=1e-4, verbose=False, warm_start=False,
                                        momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1,
                                        beta_1=0.9, beta_2=0.999, epsilon=1e-8, n_iter_no_change=10,)
        MLP_model.fit(X_train, y_train)
        train_PredictY= MLP_model.predict(X_train)
        new_feature=[Stability_Graph_Q,Stability_Graph_Jn,Stability_Graph_Ja,Stability_Graph_A,
                     Stability_Graph_B,Stability_Graph_C,Stability_Graph_HR,Stability_Graph_N]
        X_new_df = pd.DataFrame([new_feature], columns=X.columns)
        prediction = MLP_model.predict(X_new_df)[0]   
        if prediction == 0:
            MLP_prediction_category="Stable"
        if prediction == 1:
            MLP_prediction_category="Unstable"
        self.MLP_Stability_Degree.set(MLP_prediction_category)
        if not (hasattr(self, "ok_final") and hasattr(self, "lambda_star") and hasattr(self, "tau_star") and hasattr(self, "MLP_model_Boundary")):
            return
        
        hr_pt = float(Stability_Graph_HR)
        n_pt  = float(Stability_Graph_N)
        
        pstar, Lp, Up = self._pstar_LU_for_point(hr_pt, n_pt)

        tau = float(self.tau_star)
        if (Lp <= tau) and (Up >= tau):
            zone = "Uncertain"
            color = "#6c6c6c"
        elif pstar >= tau:
            zone = "Unstable"
            color = "#d62728"
        else:
            zone = "Stable"
            color = "#1f77b4"

        label_txt = f"{zone}: p*={pstar:.2f}, τ*={tau:.2f}"

        self._highlight_point_on_axes(hr_pt, n_pt, label_text=label_txt, color=color)


        
class ToolTip(object):
    def __init__(self, widget):
        self.widget = widget
        self.tipwindow = None
        self.id = None
        self.x = self.y = 0

    def showtip(self, text):
        self.text = text
        if self.tipwindow or not self.text:
            return
        x, y, cx, cy = self.widget.bbox("insert")
        x = x + self.widget.winfo_rootx() + 20
        y = y + cy + self.widget.winfo_rooty() +20
        self.tipwindow = tw = tk.Toplevel(self.widget)
        tw.wm_overrideredirect(1)
        tw.wm_geometry("+%d+%d" % (x, y))
        label = tk.Label(tw, text=self.text, justify=tk.LEFT,
                      background="#ffffe0", relief=tk.SOLID, borderwidth=1,
                      font=("tahoma", "8", "normal"))
        label.pack(ipadx=1)

    def hidetip(self):
        tw = self.tipwindow
        self.tipwindow = None
        if tw:
            tw.destroy()

In [40]:
if __name__ == '__main__':
    Main_menu()
    tk.mainloop()